# Multi-layer perceptron (classification)

In [1]:
using MLDatasets
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using Statistics: mean

## Load data

In [2]:
train_x, train_y = MNIST.traindata()
test_x,  test_y  = MNIST.testdata()

(FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

## Preprocessing

In [3]:
train_x = train_x / 255
train_x = hcat([vec(train_x[:, :, k]) for k in 1:size(train_x, 3)]...)

784×60000 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [4]:
train_y = onehotbatch(train_y, 0:9)

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

In [5]:
dataset = repeated((train_x, train_y), 200)

Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float32,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}}(Base.Iterators.Repeated{Tuple{Array{Float32,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[false true … false false; false false … false false; … ; false false … false true; false false … false false])), 200)

## Model

In [6]:
m = Chain(
    Dense(28^2, 32, relu),
    Dense(32, 10),
    softmax)

Chain(Dense(784, 32, NNlib.relu), Dense(32, 10), NNlib.softmax)

In [7]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

In [8]:
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

accuracy (generic function with 1 method)

## Optimizer

In [9]:
evalcb = throttle(() -> @show(loss(train_x, train_y)), 10)
opt = ADAM(params(m))

#43 (generic function with 1 method)

In [ ]:
@time Flux.train!(loss, dataset, opt, cb=evalcb)

loss(train_x, train_y) = 2.302221019986514 (tracked)
loss(train_x, train_y) = 2.3013972869683483 (tracked)
loss(train_x, train_y) = 2.3007412561931138 (tracked)
loss(train_x, train_y) = 2.2999150078557635 (tracked)
loss(train_x, train_y) = 2.299249396334979 (tracked)
loss(train_x, train_y) = 2.298585655026397 (tracked)
loss(train_x, train_y) = 2.2979188895926552 (tracked)
loss(train_x, train_y) = 2.2972423390909538 (tracked)
loss(train_x, train_y) = 2.2965560885557013 (tracked)
loss(train_x, train_y) = 2.295853351146745 (tracked)
loss(train_x, train_y) = 2.2951300455244983 (tracked)
loss(train_x, train_y) = 2.294191516518001 (tracked)
loss(train_x, train_y) = 2.293408340370236 (tracked)
loss(train_x, train_y) = 2.2925928532493933 (tracked)
loss(train_x, train_y) = 2.2917416594734252 (tracked)
loss(train_x, train_y) = 2.2906228747512314 (tracked)
loss(train_x, train_y) = 2.2894381315423487 (tracked)
loss(train_x, train_y) = 2.2884392945550402 (tracked)
loss(train_x, train_y) = 2.2873923

In [ ]:
accuracy(train_x, train_y)

In [ ]:
# Test set accuracy
test_x = test_x / 255
test_x = hcat([vec(test_x[:, :, k]) for k in 1:size(test_x, 3)]...)
test_y = onehotbatch(test_y, 0:9)

In [ ]:
accuracy(test_x, test_y)